# 01 - Setup & Dependencies

This notebook sets up the environment for PPO+LLM Strategy Shaping experiments.

**Run these cells in order before running other notebooks.**

In [ ]:
# Cell 1: Base setup - clone overcooked repo
%cd /content

# Get a clean fresh copy of the HumanCompatibleAI overcooked repo
!rm -rf overcooked_ai
!git clone --depth 1 https://github.com/HumanCompatibleAI/overcooked_ai.git /content/overcooked_ai > /dev/null

import sys, os
sys.path.append("/content/overcooked_ai/src")

print("CWD:", os.getcwd())
print("overcooked_ai in path:", "/content/overcooked_ai/src" in sys.path)

In [ ]:
# Cell 2: Install core RL + env deps
!pip install -q gym "stable-baselines3[extra]" overcooked-ai transformers

In [ ]:
# Cell 3: Fix numpy version (run this cell, then restart runtime)
!pip uninstall -y numpy
!pip install numpy==1.26.4

import os
os._exit(0)   # <-- Forces an immediate runtime restart

In [ ]:
# Cell 4: Pin numpy and add shimmy (sb3 + gym compat)
!pip install -q "shimmy>=2.0"

In [ ]:
# Cell 5: Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import os, zipfile

DRIVE_ROOT = "/content/drive/MyDrive"
print("Contents of MyDrive:", os.listdir(DRIVE_ROOT))

# Prefer the already-unzipped runs folder if it exists
RUNS_ROOT = os.path.join(DRIVE_ROOT, "runs")

# If runs folder is not there but runs.zip is, unzip once
if (not os.path.exists(RUNS_ROOT)) and os.path.exists(os.path.join(DRIVE_ROOT, "runs.zip")):
    print("Unzipping runs.zip into MyDrive...")
    with zipfile.ZipFile(os.path.join(DRIVE_ROOT, "runs.zip"), "r") as zf:
        zf.extractall(DRIVE_ROOT)

print("Using RUNS_ROOT:", RUNS_ROOT, "exists:", os.path.exists(RUNS_ROOT))

In [ ]:
# Cell 6: Verify installation
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from transformers import AutoTokenizer, AutoModelForCausalLM

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

print("\n✅ Setup complete! You can now run the other notebooks.")